In [86]:
import numpy as np
import random as rnd
import os
import re
from IPython.display import HTML, display
import matplotlib.pyplot as plt

In [87]:
languages = ['java_11_hotspot', 'java_11_openj9', 'java_8_hotspot', 'java_8_openj9', 'graal', 'nodejs', 'nodejs_chakra', 'go', 'go_gccgo', 'posix', 'posix_clang', 'posixmt', 'posixmt_clang']
fig_counter = 0
languages_count = len(languages)
all_syscalls_count = 385 #see https://github.com/torvalds/linux/blob/master/arch/x86/entry/syscalls/syscall_64.tbl
root='../../../target/logs'

In [88]:
def load_syscalls(filename):
    file = open(filename)
    while True:
        line = file.readline()      
        if not line:
            break
        if not line.startswith(' '):
            continue
        line = ' '.join(line.split()).split() #replaces multiple blank spaces by single spaces, and split
        yield line[-1]

def load_syscall_invokations(filename):
    file = open(filename)
    while True:
        line = file.readline()      
        if not line:
            break
        if not line.startswith(' '):
            continue
        line = ' '.join(line.split()).split() #replaces multiple blank spaces by single spaces, and split
        yield int(line[3])
        
        
all_used_syscalls = []
for language in languages:
    syscalls = np.array(list(load_syscalls(root + '/' + language + '/strace.log')))
    all_used_syscalls.extend(syscalls)
#     print(str(syscalls) + ":" + str(len(syscalls)))

all_used_syscalls = set(all_used_syscalls)

common_syscalls = all_used_syscalls.copy()
for language in languages:
    syscalls = np.array(list(load_syscalls(root + '/' + language + '/strace.log')))
    common_syscalls = common_syscalls & set(syscalls)
    
all_used_syscalls = list(all_used_syscalls)    

incidence = len(all_used_syscalls)/all_syscalls_count


# SIMULATION 1: 

We assume syscall fail one by one, permanently.
If a language relies on a failed syscall, it is killed.
The simulation ends when all used syscalls have been failed, or when all languages have been killed.

In [204]:
print(str(len(all_used_syscalls)) + ' (' + str(incidence) + ')')
print(str(len(common_syscalls)) + ' (' + str(len(common_syscalls)/all_syscalls_count) + ')')

step = 0
killed = []
max_step = 1000
syscall_index = 0

rnd_used_syscalls = all_used_syscalls.copy()
rnd.shuffle(rnd_used_syscalls)
while len(killed) < languages_count and step < max_step and syscall_index < len(all_used_syscalls):
    r = np.random.uniform(0,1)
    if r <= incidence: #one syscall in all_used_syscalls is failed
        syscall = rnd_used_syscalls[syscall_index]
        new_killed = False
        for language in languages:
            if not language in killed:
                syscalls = np.array(list(load_syscalls(root + '/' + language + '/strace.log')))
                if syscall in syscalls:
                    killed.append(language)
                    new_killed = True
        if new_killed:
            print(killed)
        syscall_index = syscall_index + 1
    step = step + 1
    
print('steps: ' + str(step))
# print('killed: ' + str(killed))

78 (0.2025974025974026)
11 (0.02857142857142857)
['java_11_openj9', 'java_8_openj9', 'nodejs_chakra']
['java_11_openj9', 'java_8_openj9', 'nodejs_chakra', 'java_11_hotspot', 'java_8_hotspot', 'nodejs', 'go', 'go_gccgo']
['java_11_openj9', 'java_8_openj9', 'nodejs_chakra', 'java_11_hotspot', 'java_8_hotspot', 'nodejs', 'go', 'go_gccgo', 'graal']
['java_11_openj9', 'java_8_openj9', 'nodejs_chakra', 'java_11_hotspot', 'java_8_hotspot', 'nodejs', 'go', 'go_gccgo', 'graal', 'posix', 'posix_clang', 'posixmt', 'posixmt_clang']
steps: 55
